In [4]:
import openpyxl
import os
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
import tkinter as tk

In [22]:
# добавление к отчетному файлу сообщения
def add_report(message):
    with open('report.txt', 'a') as report_file:
        report_file.write(message)


class DataFileXLS:

    def __init__(self, file_name):
        self.file_name = file_name
        self.fact = 'факт'
        self.plan = 'план'
    

    # функция открывает файл и возвращает первую его страницу
    def open_file(self):
        self.file = openpyxl.load_workbook(f'{self.file_name}')
        self.sheet = self.file.active
        return self.sheet


    # определение координат начальной и конечной ячеек данных с человеко-часами
    def min_cell(self, sheet, max_col, max_row):
        for col in range(1,max_col):
            for row in range(1,max_row):
                if self.isfloat(sheet.cell(row=row, column=col).value):
                    return sheet.cell(row=2, column=col).coordinate 
    
    def max_cell(self, sheet, max_row, max_col):
        return sheet.cell(row=max_row, column=max_col).coordinate

    
    # чтение файла построчно
    def read_file_by_line(self, sheet, min_cell, max_cell):
        lst_file_by_line = list()
        for rows in sheet[min_cell:max_cell]:
            lst_in = list()
            for cell in rows:
                # значение ячейки
                cell_val = cell.value

                # замена None на ''
                if cell_val == None:
                    cell_val = ''
                
                lst_in.append(cell_val)
                
            lst_file_by_line.append(lst_in)
        
        return lst_file_by_line


    # зануление строк с дублирующимися проектами
    def destroy_dublicates(self, max_row, lst_file_by_line, sheet, max_col, min_cell):

        lst_projects = list()
        for r in range(2,max_row):
            lst_projects.append(sheet.cell(row=r, column=1).value)

        # приведение названия проекта к универсальному виду "№ проект"
        for i in range(len(lst_projects)):
            lst_projects[i] = ' '.join(sorted(lst_projects[i].split()))

        for project in set(lst_projects):
            # если название проекта дублируется
            if lst_projects.count(project) > 1:
                # номера строк дублирующегося проекта
                index_project = [i for i, proj in enumerate(lst_projects) if proj == project]
                
                # убедиться, что строки совпадают полностью:
                duplicate = True
                for i in range(len(index_project)-1):
                    for col in range(column_index_from_string(coordinate_from_string(min_cell)[0]),max_col-1):
                        # если обнаруживается несовпадение, проверка прекращается с отрицательным результатом
                        if sheet.cell(row=index_project[i]+2,column=col).value != \
                            sheet.cell(row=index_project[i+1]+2,column=col).value:
                            duplicate = False
                            break

                # зануление каждого элемента каждой дублирующейся строки, кроме первой, для данного проекта
                if duplicate == True:
                    for index in index_project[1:]:
                        for i in range(len(lst_file_by_line[index])):
                            lst_file_by_line[index][i] = 0
        
        return lst_file_by_line


    # значения ячеек по столбцам
    def column_values(self, lst_file_by_line):
        lst_column_values = list()
        
        for i in range(len(lst_file_by_line[0])):
            lst_column_in = list()
            
            for j in range(len(lst_file_by_line)):
                lst_column_in.append(lst_file_by_line[j][i])
            
            lst_column_values.append(lst_column_in)
        
        return lst_column_values
    

    # объединение значений по два столбца
    def values_to_couple_PlanFact(self, lst_column_values):
        lst_couple = list()
        for i in range(0,len(lst_column_values),2):
            lst_couple.append(lst_column_values[i:i+2])
        
        return lst_couple


    # относится ли строка к типу float
    def isfloat(self, s):
        try:
            float(s)
            return True
        except:
            return False
    

    # получение списка успешностей сотрудников
    def difference(self, lst_couple, max_row):
        
        # success_workers_lst - список, содержащий успешность 
        # каждого сотрудника в порядке их записи в файле
        success_workers_lst = list()

        # элементы списка difference_lst - разница между планом
        # и фактом для каждого проекта для одного из сотрудников,
        # деленная на число строк с человеко-часами
        difference_lst = list()

        for i in range(len(lst_couple)):
            for j in range(len(lst_couple[0][0])):
                if self.isfloat(lst_couple[i][0][j]) and self.isfloat(lst_couple[i][1][j]):
                    difference_lst.append((float(lst_couple[i][0][j])-
                        float(lst_couple[i][1][j]))/(max_row-1))

            # присоединение разности к списку
            success_workers_lst.append(sum(difference_lst))

            # очистка списка перед тем, как приступить к обработке
            # успешности следующего сотрудника
            difference_lst.clear()
        
        return success_workers_lst


    # данные в ячейках, относящиеся к одному сотруднику, должны
    # совпадать и быть корректными
    def ispersons(self, person1, person2):
        person1, person2 = person1.lower(), person2.lower()
        # проверка для каждой ячейки, что ФИО в нужном формате
        persons = list()
        for person in [person1, person2]:
            person_FIO = (person).split()
            if 'план' in person_FIO:
                person_FIO.remove('план')
            elif 'факт' in person_FIO:
                person_FIO.remove('факт')
            persons.append(person_FIO)

            # должно быть два элемента - фамилия и И.О.
            # также проверяется, не содержит ли фамилия лишних символов
            if len(person_FIO) != 2 or not person_FIO[0].isalpha():
                return False
                        
            # исследуются И.О.
            person_IO = person_FIO[1].split('.')[:-1]
            if len(person_IO) == 2:
                for elem in person_IO:
                    if not elem.isalpha():
                        return False
            else:
                return False

        if 'факт' in person1 or 'план' in person2:
            add_report(f'Перепутаны столбцы у сотрудника {(" ".join(persons[0])).title()} '
                f'в файле {self.file_name}!'+'\n')
        
        # совпадают ли данные ячеек
        if persons[0] != persons[1]:
            return False
                
        return True


    # функция для удаления слов "Факт" или "План"
    # для корректного отображения ФИО сотрудника
    def correct_name_person(self, person):
        person = (person.lower()).split()
        if 'план' in person:
            person.remove('план')
        elif 'факт' in person:
            person.remove('факт')
        
        return (' '.join(person)).title()


    # получение имён сотрудников
    def name_workers(self, sheet, max_col, min_cell):
        workers = list()

        # номер столбца первой ячейки с человеко-часами
        col_min_cell = column_index_from_string(coordinate_from_string(min_cell)[0])
        
        for col in range(col_min_cell,max_col+1,2):
            person1 = (sheet.cell(row=1, column=col).value)
            person2 = (sheet.cell(row=1, column=col+1).value)
            
            if self.ispersons(person1, person2):
                person1 = self.correct_name_person(person1)
                workers.append(person1)
            else:
                person1 = self.correct_name_person(person1)
                person2 = self.correct_name_person(person2)

                add_report(f'Некорректные данные в файле {self.file_name} в ячейке '
                    f'{sheet.cell(row=1, column=col).coordinate} '
                    f'или {sheet.cell(row=1, column=col+1).coordinate}, '
                    f'сотрудник {person1}/{person2} не учтён!'+'\n')
        
        return workers


    # успешность каждого сотрудника поимённо
    def success_dict(self, workers, success_workers_lst):
        success_workers_dict = dict()
        for i in range(len(workers)):
            success_workers_dict[workers[i]] = success_workers_lst[i]
        
        return success_workers_dict
    

# ------------------------------------- #
#     функции для обработки файлов      #
# ------------------------------------- #

# обработка одного файла
def success_by_one_file(file_name):
    
    file = DataFileXLS(file_name)
    sheet = file.open_file()

    max_row = file.sheet.max_row
    max_col = file.sheet.max_column
    min_cell =  file.min_cell(sheet, max_col, max_row)
    max_cell = file.max_cell(sheet,max_row,max_col)

    lst_file_by_line = file.read_file_by_line(sheet, min_cell, max_cell)
    lst_file_by_line = file.destroy_dublicates(max_row, lst_file_by_line, sheet, max_col, min_cell)

    lst_column_values = file.column_values(lst_file_by_line)
    lst_couple = file.values_to_couple_PlanFact(lst_column_values)
    success_workers_lst = file.difference(lst_couple, max_row)
    workers = file.name_workers(sheet, max_col, min_cell)
    success_workers_dict = file.success_dict(workers, success_workers_lst)

    return success_workers_dict


# совмещение данных из файлов в один словарь
def combination_dicts(lst_dicts):

    # содержит фио сотрудника и его успешность
    success_workers_dict_all_files = dict()

    # множество позволит не обрабатывать одних 
    # и тех же сотрудников несколько раз
    unic_workers = set()

    # перебор словарей
    for i in range(len(lst_dicts)):

        # key_i - рассматриваемый сотрудник
        for key_i in lst_dicts[i].keys():
            # values_lst - список успешностей для одного сотрудника из разных файлов
            values_lst = [lst_dicts[i].get(key_i)]

            # остальные словари
            for j in range(i+1, len(lst_dicts)):

                # если сотрудник упоминается в другом файле
                if key_i in lst_dicts[j].keys():
                    unic_workers.add(key_i)
                    values_lst.append(lst_dicts[j].get(key_i))
                
                # к итоговому словарю добавляем пару фио сотрудника - его успешность
                success_workers_dict_all_files[key_i] = sum(values_lst) 

            # если сотрудник есть только в одном файле,
            # его успешность добавляется без изменений
            if key_i not in unic_workers:
                success_workers_dict_all_files[key_i] = lst_dicts[i].get(key_i)
    
    return success_workers_dict_all_files


# обобщение на все файлы директории
def success_all_files():

    lst_dicts = list()
    # чтение всех файлов из директории и запись сотрудников и их
    # успешностей по отдельным словарям в список lst_dicts
    for elem in os.listdir():
        if elem[-4:] == 'xlsx' or elem[:-3] == 'xls':
            lst_dicts.append(success_by_one_file(elem))

    success_workers_dict_all_files = combination_dicts(lst_dicts)

    # печать сотрудников по успешности
    add_report('\nКорректно обработанные сотрудники в порядке убывания успешности:\n')
    #print(success_workers_dict_all_files)
    for elem in sorted(success_workers_dict_all_files, 
            key=success_workers_dict_all_files.get, reverse=True):
        add_report(elem+'\n')


# --------------------------------------- #
#   окно с вопросом, открывать ли отчёт   #
# --------------------------------------- #

class WinAskForOpenReport(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title('Отчёт')

        # размеры диалогового окна
        self.geometry("264x90")
        self.grid_rowconfigure(0,minsize=50)

        # сообщение в окне
        label1 = tk.Label(text="Обработка файлов завершена. Открыть отчёт?")
        label1.grid(column=0, row=0, columnspan=2)

        # кнопки "да" и "нет"
        btn_yes = tk.Button(self, text='Да', command=self.open_report_file)
        btn_no = tk.Button(self, text='Нет', command=self.destroy)
        btn_yes.grid(column=0, row=1, stick='we')
        btn_no.grid(column=1, row=1, stick='we')

    # если "да"
    def open_report_file(self):
        os.startfile('report.txt')
        self.destroy()


# --------------------------------------------------------------------------- #
#                                                                             #
#                    начало выполнения программы                              #
#                                                                             #
# --------------------------------------------------------------------------- #

# создание/очистка файла перед выполнением программы
open('report.txt','w').close()

try:
    success_all_files()
except ValueError:
    add_report('Некорректные данные в ячейках с человеко-часами!\n')
except PermissionError:
    add_report('Необходимо закрыть все файлы перед их обработкой!\n')
except IndexError:
    add_report('Один из файлов содержит некорректное количество столбцов!\n')
except AttributeError:
    add_report('Некорректная структура страницы одного из файлов!\n')

# диалоговое окно
root = WinAskForOpenReport()
root.mainloop()